In [ ]:
import numpy as np
import pandas as pd 
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('vader_lexicon')

In [ ]:
import csv
df = pd.read_csv(r'/Users/rohan/Desktop/NLP_Group_7/FullyProcessedDataset.csv')


file_path = '/Users/rohan/Desktop/NLP_Group_7/song_topic_distribution.csv'

def convert_csv_to_list_of_tuples(file_path):
 
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  

        column_numbers = [int(re.search(r'\d+', col).group()) for col in header if re.search(r'\d+', col)]

        result = []

        for row in reader:
        
            row_data = [(column_numbers[i], float(value)) for i, value in enumerate(row) if float(value) != 0.0]
            result.append(row_data)
        return result


song_topic_distribution = convert_csv_to_list_of_tuples(file_path)


In [ ]:
def RecommendSongs(song_topic_distribution, user_input):

    try: 

        user_song = (df.index[df['SName'] == user_input]).item()

    #################################################################################

        def calculate_hellinger_distance(song_dist_1, song_dist_2):
            return hellinger(song_dist_1, song_dist_2)

        input_song_index = user_song

        input_song_dist = song_topic_distribution[input_song_index]

        hellinger_distances = []

        for i, song_dist in enumerate(song_topic_distribution):

            distance = calculate_hellinger_distance(input_song_dist, song_dist)
            hellinger_distances.append((distance))

        df['HDF'] = hellinger_distances

    #################################################################################

        userGenre = df['Genres'].iloc[user_song]
        genreReducedDf = df.loc[df['Genres'] == userGenre]
        genreReducedDf.reset_index(inplace = True)
        userSongIndexUpdated = (genreReducedDf.index[genreReducedDf['SName'] == user_input]).item()

    #################################################################################

        analyzer = SentimentIntensityAnalyzer()

        def get_sentiment_score(lyrics):
            sentiment = analyzer.polarity_scores(lyrics)
            sentiment_score = sentiment['compound']
            return sentiment_score

        sentiment = get_sentiment_score(genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated])

        SSF_list = []
        for i in range(genreReducedDf.shape[0]):
            SSF_list.append(abs(genreReducedDf['Sentiment'].iloc[i] - sentiment))

        genreReducedDf['SSF'] = SSF_list

    #################################################################################

        tfidf_vectorizer = TfidfVectorizer()
        CSF_list = []
        for i in range(genreReducedDf.shape[0]):
            lyrics_matrix = tfidf_vectorizer.fit_transform([genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated], genreReducedDf['Lemmatized_Lyrics'].iloc[i]])
            lyrics_similarity = cosine_similarity(lyrics_matrix)
            CSF_list.append(1 - (lyrics_similarity[0][1]))
        genreReducedDf['CSF'] = CSF_list

    #################################################################################

        genreReducedDf['SF'] = genreReducedDf['SSF'] + genreReducedDf['CSF'] + genreReducedDf['HDF']
        genreReducedDf = genreReducedDf.nsmallest(6,["SF"])
        genreReducedDf.drop(userSongIndexUpdated, axis = 0, inplace = True)

    #################################################################################

        RecommendationDF = genreReducedDf[['Artist', 'SName']]
        RecommendationDF.reset_index(inplace = True)
        RecommendationDF = RecommendationDF.drop('index', axis = 1)
        RecommendationDF.columns = ['ARTIST', 'SONG TITLE']

    #################################################################################

        return RecommendationDF
    
    except:

        ErrorDf = pd.DataFrame({'Error': ['Song Not Found']})
        return ErrorDf
